### Citibike Analysis

In [1]:
import os
from dotenv import load_dotenv
from google.cloud import bigquery


In [2]:
load_dotenv()

True

In [3]:
client = bigquery.Client()

bq_project_id = os.getenv('GCP_PROJECT_ID')
bq_dataset_name = os.getenv('BQ_DATASET')
bq_table_name = os.getenv('BQ_TABLE')

In [4]:
query = f"""
    SELECT *
    FROM `{bq_project_id}.{bq_dataset_name}.202401-{bq_table_name}`
    LIMIT 10
"""

df = client.query(query).to_dataframe()
df.head()

/home/olshe/pers/eu-climate-economic-resilience/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,03DF018B20ED02D9,classic_bike,2024-01-13 14:51:04.773,2024-01-14 15:50:59.897,Columbia St & Kane St,4422.05,None,nan,40.687632,-74.001626,NaN,NaN,casual
1,3E194E3EE8189D43,classic_bike,2024-01-03 16:09:56.876,2024-01-03 16:17:52.884,E 72 St & York Ave,6889.12,E 74 St & 1 Ave,6953.08,40.766638,-73.953483,40.768974,-73.954823,casual
2,33658F1AE88A8D58,classic_bike,2024-01-24 08:14:57.910,2024-01-24 08:31:02.612,8 Ave & W 27 St,6297.07,W 10 St & Washington St,5847.06,40.747968,-73.996637,40.733424,-74.008515,casual
3,49D93C7E1ED3CEB6,classic_bike,2024-01-08 14:59:44.611,2024-01-09 15:59:36.472,Parkside Ave & Ocean Ave,3448.02,None,nan,40.654850,-73.962170,NaN,NaN,casual
4,6462D9ABC53AA8BE,classic_bike,2023-12-31 16:28:05.221,2024-01-01 17:27:58.513,Catalpa Ave & Woodward Ave,4912.10,None,nan,40.702290,-73.901450,NaN,NaN,casual
